In [109]:
from pyspark.sql import SparkSession 

sc = SparkSession.builder.appName("DataFrame").getOrCreate()

In [110]:
logs_interesse = [
    "Eleitor foi habilitado",
    "Voto confirmado para [Deputado Federal]",
    "Voto confirmado para [Deputado Estadual]",
    "Voto confirmado para [Senador]",
    "Voto confirmado para [Governador]",
    "Voto confirmado para [Presidente]",
    "Tecla indevida pressionada",
]
regex_interesse = [    "Município:",
    "Zona Eleitoral:",
    "Local de Votação:",
    "Seção Eleitoral:"]
regex_interesse = "|".join(regex_interesse)

In [111]:
import pyspark.sql.functions as F

DATA_PATH = "/app/data/"
input_file = DATA_PATH+"decodificado/*" 

df = sc.read.csv(input_file, sep="\t", header=False)
df = df.withColumnRenamed("_c0","data") \
    .withColumnRenamed("_c4","descricao")
df = df.select("data","descricao")
df = df.filter(F.col("descricao").isin(logs_interesse) | F.col("descricao").rlike(regex_interesse))
df = df.withColumn("data", F.to_timestamp(F.col("data"), 'dd/MM/yyyy HH:mm:ss'))

df.show(truncate=False)

+-------------------+----------------------+
|data               |descricao             |
+-------------------+----------------------+
|2022-09-22 15:15:23|Município: 18236      |
|2022-09-22 15:15:23|Zona Eleitoral: 0016  |
|2022-09-22 15:15:23|Local de Votação: 1120|
|2022-09-22 15:15:23|Seção Eleitoral: 0158 |
|2022-09-22 15:22:45|Município: 18236      |
|2022-09-22 15:22:45|Zona Eleitoral: 0016  |
|2022-09-22 15:22:45|Seção Eleitoral: 0158 |
|2022-09-22 15:35:49|Município: 18236      |
|2022-09-22 15:35:49|Zona Eleitoral: 0016  |
|2022-09-22 15:35:49|Seção Eleitoral: 0158 |
|2022-09-28 14:51:55|Município: 18236      |
|2022-09-28 14:51:55|Zona Eleitoral: 0016  |
|2022-09-28 14:51:55|Seção Eleitoral: 0158 |
|2022-10-01 15:24:28|Município: 18236      |
|2022-10-01 15:24:28|Zona Eleitoral: 0016  |
|2022-10-01 15:24:28|Seção Eleitoral: 0158 |
|2022-10-02 07:26:05|Município: 18236      |
|2022-10-02 07:26:05|Zona Eleitoral: 0016  |
|2022-10-02 07:26:05|Seção Eleitoral: 0158 |
|2022-10-0

In [113]:
from pyspark.sql.window import Window
from unidecode import unidecode

window = (
    Window
    .orderBy('data')
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)
)

for nome_extracao in ["Município","Zona Eleitoral","Seção Eleitoral"]:
     nome_col = unidecode(nome_extracao).replace(" ","_")
     df = df.withColumn(nome_col,
                    F.when(F.col("descricao").contains(nome_extracao),
                         F.regexp_extract(F.col("descricao"), rf"{nome_extracao}: (\d+)", 1).cast("integer"))
                    .otherwise(None))

     df = (
     df
     .withColumn(nome_col, F.last(nome_col, ignorenulls=True).over(window))
     )

In [114]:
df = df.filter(~F.col("descricao").rlike(regex_interesse))
#df = df.dropDuplicates()
df = df.withColumn("erros_tecla",
                   F.when(F.col("descricao") == "Tecla indevida pressionada", 1)
                   .otherwise(0))

df = df.withColumn("id_eleitor",
                   F.when(F.col("descricao") == "Eleitor foi habilitado", 1)
                   .otherwise(0))
my_window = (Window.orderBy('data')
             .rowsBetween(Window.unboundedPreceding, 0))
df = df.withColumn('id_eleitor', F.sum('id_eleitor').over(my_window))

df.show(truncate=False)

24/03/13 21:39:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/13 21:39:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/13 21:39:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/13 21:39:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/13 21:39:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/13 21:39:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/13 2

+-------------------+----------------------------------------+---------+--------------+---------------+-----------+----------+
|data               |descricao                               |Municipio|Zona_Eleitoral|Secao_Eleitoral|erros_tecla|id_eleitor|
+-------------------+----------------------------------------+---------+--------------+---------------+-----------+----------+
|2022-10-02 08:01:06|Eleitor foi habilitado                  |18236    |16            |139            |0          |1         |
|2022-10-02 08:01:07|Eleitor foi habilitado                  |18236    |16            |139            |0          |2         |
|2022-10-02 08:01:07|Eleitor foi habilitado                  |18236    |16            |139            |0          |3         |
|2022-10-02 08:01:11|Tecla indevida pressionada              |18236    |16            |139            |1          |3         |
|2022-10-02 08:01:14|Voto confirmado para [Deputado Federal] |18236    |16            |139            |0       

In [80]:
df.printSchema()

root
 |-- data: timestamp (nullable = true)
 |-- descricao: string (nullable = true)
 |-- Municipio: integer (nullable = true)
 |-- Zona_Eleitoral: integer (nullable = true)
 |-- Secao_Eleitoral: integer (nullable = true)



In [ ]:
sc.stop()